**This Notebook is to explain what is behind pipeline**

#pipline

In [2]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")
raw_inputs = [
              "I've been waiting for a HuggingFace course my whole life.",
              "I hate this so much!",
            ]
classifier(raw_inputs)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

[{'label': 'POSITIVE', 'score': 0.9598048329353333},
 {'label': 'NEGATIVE', 'score': 0.9994558691978455}]

#Steps behind pipeline

###Tokenization

In [5]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

raw_inputs = [
              "I've been waiting for a HuggingFace course my whole life.",
              "I hate this so much!",
            ]
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662, 12172,
          2607,  2026,  2878,  2166,  1012,   102],
        [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,     0,
             0,     0,     0,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}


###Base Model <br>
AutoModel is the base model, which is originally provided by Bert. All task specific models add layers on top of that to customze for the task, the added layers are called head.

In [17]:
from transformers import AutoModel

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(model_name)
outputs = model(**inputs)
print(outputs.last_hidden_state.shape)

'''
AutoModel contains only the base Transformer module -- encoder.
The output is the last hidden state, which is a high-dimensional vector representing the contextual enrichments from input by the encoder.
The output is usually inputs to topper layers (known as the head) of task-specific model which is built based on the base model.
Different tasks could have been performed with the same base model, but will have a different head associated with them.

output shape is [2, 16, 768]
2 is the batch size, because we have two sentences in raw_inputs
16 is the length of sequence predefined. Longer sequences will be truncated, shorter sequence will be padded.
768 is from the word embedding vector lenght.
'''

torch.Size([2, 16, 768])


'\nAutoModel contains only the base Transformer module -- encoder.\nThe output is the last hidden state, which is a high-dimensional vector representing the contextual enrichments from input by the encoder.\nThe output is usually inputs to topper layers (known as the head) of task-specific model which is built based on the base model.\nDifferent tasks could have been performed with the same base model, but will have a different head associated with them.\n\noutput shape is [2, 16, 768]\n2 is the batch size, because we have two sentences in raw_inputs\n16 is the length of sequence predefined. Longer sequences will be truncated, shorter sequence will be padded.\n768 is from the word embedding vector lenght.\n'

###Head <br>
In the below code, AutoModelForSequenceClassification incudes the base model, on top of that, it add task specific layer known as head. The output is logits. <br>
All Transformers models output the logits, we need last activation funciton such as SoftMax to complete the final step.

In [23]:
from transformers import AutoModelForSequenceClassification

model_name = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
outputs = model(**inputs)
print(outputs)

'''
outputs are logits, which is the raw, unnormalized output by the last layer of the model AutoModelForSequenceClassification

ouput.logits = tensor([[-1.5607,  1.6123],[ 4.1692, -3.3464]]
[-1.5607, 1.6123] for the first sentenc
[ 4.1692, -3.3464] for the second one.
'''

SequenceClassifierOutput(loss=None, logits=tensor([[-1.5607,  1.6123],
        [ 4.1692, -3.3464]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


### Output activation function
Finally, apply output activation funciton to tranform output logits to preditions.

In [29]:
import torch
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)             # output probabilities of Positive and Negative
print(model.config.id2label)   # how the index is predefined with labels within the model

tensor([[4.0195e-02, 9.5980e-01],
        [9.9946e-01, 5.4418e-04]], grad_fn=<SoftmaxBackward0>)
{0: 'NEGATIVE', 1: 'POSITIVE'}
